# Testing

# What is testing?

* TODO

# The `unittest` module 

* xUnit similar to jUnit
* each test case is a class inherting from `unittest.TestCase`
* test runner executes all `test_xxx` method
* test methods can `assertXXX()` conditions
* test results can be printed, stored as XML etc.

## A simple function to test

In [1]:
def divided(dividend, divisor):
    return dividend / divisor

## The test case

In [2]:
# Simple test case to test a trivial ``divided`` function.
import unittest


class DivideTest(unittest.TestCase):
    def test_can_divide_positive_numbers(self):
        self.assertEqual(3, divided(15, 5))
        self.assertAlmostEqual(2.5, divided(5, 2))

    def test_can_divide_negative_numbers(self):
        self.assertEqual(-3, divided(15, -5))

    def test_can_divide_zero(self):
        self.assertEqual(0, divided(0, 1))
        self.assertEqual(0, divided(0, -1))
        self.assertEqual(0, divided(0, 123.45))

## Testing for expected errors

In [4]:
class DivideTest(unittest.TestCase):
    # ...
    def test_fails_on_zero_division(self):
        self.assertRaises(ZeroDivisionError, divided, 1, 0)

Note that `divided()` is not called here, only the function and its parameters is passed.

## Common `assertXXX()` methods

* `asserEqual(a, b)` - check that `a == b`
* `assertAlmostEqual(a, b)` - check that two floating point numbers have almost the same value
* `assertRegex(text, regex)` - check that a `text` matches a regular expression
* `assertIn(needle, haystack)` - check that `needle` is in `haystack`
* `assertIsNone(value)` - check that `value` is `None`
* `assertTrue(expression)`, `assertFalse(expression)` - check the result of any expression

## Common `assertXXX()` methods (continued)

All methods have an optional argument `message` to specify your own message in case of error. Typically the standard messages are adequate though with the exception of `assertTrue()` and `assertFalse()`.

Most methods also have a `Not` equivalent, e.g. `assertEqual()` and `assertNotEqual()`.

For a full list, visit https://docs.python.org/3/library/unittest.html.

## Test runners

* Test runners look for classes inheriting from `unittest.TestCase` (or children of it) and run all `test_xxx(self)` methods in it.
* Some test runners also can recursively discover test cases across multiple folders.
* If a `test_xxx()` fails, the test runner continues with test next test method.
* The test runner collects all results.

## Let's break things

Change the division operator (`/`) to integer division (`//`) to break one of our test cases, so the test results get more interesting:

In [6]:
def divided(dividend, divisor):
    return dividend // divisor

## Command line test runner

The `unittest` module already provides a built in test runner that can be called from the command line by simply adding the following lines to the test case:

```python
if __name__ == '__main__':
    unittest.main()
```

## Command line test runner: example output

```
.F..
===================================================
FAIL: test_can_divide_positive_numbers (__main__.DivideTest)
---------------------------------------------------
Traceback (most recent call last):
  File "/home/roskakori/workspace/talks/python_for_testers/examples/test_divide.py", line 12, in test_can_divide_positive_numbers
    self.assertAlmostEqual(2.5, divided(5, 2))
AssertionError: 2.5 != 2 within 7 places

---------------------------------------------------
Ran 4 tests in 0.001s

FAILED (failures=1)
```

## IDE test runners

![Screenshot: PyCharm test runner](examples/pycharm_test_runner.png)

## Failures and errors

* TODO

# Test fixtures

* TODO

# The `pytest` module

* TODO

# Summary

* TODO